In [1]:
import numpy as np
import copy
from lxml.etree import Element, SubElement, tostring, ElementTree
from xml.dom import minidom 
import cv2
import os
import os.path

In [2]:
# 读入路径
xml_path = r'/Users/edz/Documents/yandi_data/taishi_loading/to_xlm/xml_path/'  # 创建的xml文件路径
# 图片文件夹
image_path = r'/Users/edz/Documents/yandi_data/taishi_loading/'  # 原始图片路径

### 病灶标签的对照关系¶
0:id12, 1:id13, 2:id14, 3:id15, 4:id18, 5:id19, 6:id26

In [3]:
def id_map_func(rd_id_list):
################ 将算法输出的id值转移为真实的图片标注框id值
    # 标签ID数据集
    lb_list = []   # 存放真实id标签
    # 定义标注框ID值得映射关系————————已知算法输出VS真实ID值
    id_list = ['id12', 'id13', 'id14', 'id15', 'id18', 'id19', 'id26']  # 目标病灶ID值
    # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
    # enumerate(sequence, [start=0])
    index = { i:k for i,k in enumerate(id_list, 0) }  # 调换i:k的顺序可以改变映射关系
    print("基于原始ID值定义的映射关系：",index)
    [i for i,x in enumerate(rd_id_list) if x in index]
    print("算法输出mun值：",rd_id_list)
    lb_list = list(map(index.get, rd_id_list))
    print("对应于ID值的映射结果:",lb_list) ## list(filter(None, map(index.get, random_list)))  # 会过滤掉0
    return lb_list

In [4]:
def txt_file_func(txt_image, txt_label_list, txt_boxes):
    for i in range(txt_boxes.shape[0]):
        temp_text = []
        temp_image_i = txt_image  # 处理的图片名称——根据文件名实时修改
        temp_label_i = txt_label_list[i]
        temp_box_i = txt_boxes[i][:4].tolist()  # 截取前四位——即：标注框的坐标数据
        temp_text.append(temp_image_i)
        temp_text.append(temp_label_i) 
        temp_text = temp_text + temp_box_i  # list相加
#         print(temp_text)
        if i == 0:
            temp_txt_total = np.array(temp_text)
        else:
            temp_txt_total = np.vstack((temp_txt_total, temp_text))  #np.vstack((a,b)) 垂直组合
    print("最后生成的TXT数据文件：" ,temp_txt_total)
    return temp_txt_total

In [5]:
### 修改参数
images_folder = "to_xlm"   # 存放处理图片的文件夹
image_name = '01638.jpg'   # 处理的图片名称
# 坐标位置数据集
# 前四位数据时标注框的坐标值，最后一位坐标为输出的置信度
boxes = np.array([[1.3418148e+03, 6.2916492e+02, 1.4483580e+03, 9.2253162e+02,
        6.3271374e-01],
       [7.9343536e+02, 7.6787390e+02, 9.2182294e+02, 8.7553210e+02,
        9.9876070e-01],
       [3.2191675e+02, 5.6280017e+02, 8.5092334e+02, 1.1540432e+03,
        9.8476392e-01]]) 
random_id_list = [2, 5, 1]   # 每次均是随机的——————算法随机输出
label_list = id_map_func(random_id_list)
temp_text_total = txt_file_func(image_name, label_list, boxes)
# 读入原始图片数据
im = cv2.imread(image_path + images_folder + '/' + temp_text_total[0][0]) # 读取图片信息,temp_text_total[0][0]为图片名称   
image_size_ys = [str(im.shape[i]) for i in range(3)]   # 输出图片的size
print('原始图片的size：',image_size_ys)
# print('image_hight', str(im.shape[0]))
# print('image_width', str(im.shape[1]))
# print('image_depth', str(im.shape[2]))

基于原始ID值定义的映射关系： {0: 'id12', 1: 'id13', 2: 'id14', 3: 'id15', 4: 'id18', 5: 'id19', 6: 'id26'}
算法输出mun值： [2, 5, 1]
对应于ID值的映射结果: ['id14', 'id19', 'id13']
最后生成的TXT数据文件： [['01638.jpg' 'id14' '1341.8148' '629.16492' '1448.358' '922.53162']
 ['01638.jpg' 'id19' '793.43536' '767.8739' '921.82294' '875.5321']
 ['01638.jpg' 'id13' '321.91675' '562.80017' '850.92334' '1154.0432']]
原始图片的size： ['1556', '1924', '3']


#### 创建xml文件

In [6]:
def xml_create_func(image_folder, image_name, image_path_ab, image_size, temp_text):
    # 如果不存在xml_path的保存文件夹，则自动创建一个，并且命名为xml_path指定的路径
    if not os.path.exists(xml_path):
        os.mkdir(xml_path)
    
    # 1.创建doc树对象
    doc = minidom.Document()
    ########## owner
    # 2.创建根节点。每次都要用doc对象来创建任何节点。
    annotation_node = doc.createElement('annotation')
    # 3.用doc对象添加根节
    doc.appendChild(annotation_node)
    # folder
    folder_node = doc.createElement('folder')
    annotation_node.appendChild(folder_node)

    folder_txt = doc.createTextNode(image_folder)   # 存放图片的文件夹"to_xlm"
    folder_node.appendChild(folder_txt)
    # filename
    filename_node = doc.createElement('filename')
    annotation_node.appendChild(filename_node)

    filename_txt = doc.createTextNode(image_name)  # 图片名称temp_text[0][0]——修改
    filename_node.appendChild(filename_txt)
    # path
    path_node = doc.createElement('path')
    annotation_node.appendChild(path_node)

    path_txt = doc.createTextNode(image_path_ab)   # 原始图片路径——修改
    path_node.appendChild(path_txt)
    ############ ones#
    # source
    source_node = doc.createElement('source')
    annotation_node.appendChild(source_node)
    database_node = doc.createElement('database')
    source_node.appendChild(database_node)

    # 也用doc创建文本节点，把文本节点（文字内容）看成子节点
    database_txt = doc.createTextNode("Unknown")
    # 用添加了文本的节点对象（看成文本节点的父节点）添加文本节点
    database_node.appendChild(database_txt)

    ############# twos#
    # size
    size_node = doc.createElement('size')
    annotation_node.appendChild(size_node)
    # width
    width_node = doc.createElement('width')
    size_node.appendChild(width_node)
    width_txt = doc.createTextNode(str(image_size[1]))  # 原始图片的width
    width_node.appendChild(width_txt)
    # height
    height_node = doc.createElement('height')
    size_node.appendChild(height_node)
    height_txt = doc.createTextNode(str(image_size[0]))  # 原始图片的height
    height_node.appendChild(height_txt)
    # depth
    depth_node = doc.createElement('depth')
    size_node.appendChild(depth_node)
    depth_txt = doc.createTextNode(str(image_size[2]))   # 原始图片的depth
    depth_node.appendChild(depth_txt)
    ############### twoe#
    segmented_node = doc.createElement('segmented')
    annotation_node.appendChild(segmented_node)
    segmented_txt = doc.createTextNode("0")
    segmented_node.appendChild(segmented_txt)
    ############### three# 循环创建多个标注框
    for i in range(temp_text.shape[0]):  # temp_text.shape[0]表征打标框的个数:
        # 每一个标注框的内容更新
        txt_row = temp_text[i]  # 截取每一行全量数据
        txt_file_name = txt_row[0]   ## 图片名称——txt_file_name
        txt_name = txt_row[1]    # 标注框ID值
        txt_xmin = txt_row[2]    # 标注框xmin
        txt_ymin = txt_row[3]    # 标注框ymin
        txt_xmax = txt_row[4]    # 标注框xmax
        txt_ymax = txt_row[5]    # 标注框ymax

        # object
        object_new = doc.createElement('object')
        annotation_node.appendChild(object_new)

        #name
        name_node = doc.createElement('name')
        object_new.appendChild(name_node)
        name_txt = doc.createTextNode(txt_name)  # 对应标注框的ID值——多个打标框的情况，逐个修改ID值
        name_node.appendChild(name_txt)
        # pose
        pose_node = doc.createElement('pose')
        object_new.appendChild(pose_node)
        pose_txt = doc.createTextNode("Unspecified")
        pose_node.appendChild(pose_txt)
        # truncated
        truncated_node = doc.createElement('truncated')
        object_new.appendChild(truncated_node)
        truncated_txt = doc.createTextNode("0")
        truncated_node.appendChild(truncated_txt)
        # difficult
        difficult_node = doc.createElement('difficult')
        object_new.appendChild(difficult_node)
        difficult_txt = doc.createTextNode("0")
        difficult_node.appendChild(difficult_txt)
        ########### threes-1#
        # bndbox
        bndbox = doc.createElement('bndbox')
        object_new.appendChild(bndbox)
        # xmin
        xmin_node = doc.createElement('xmin')
        bndbox.appendChild(xmin_node)
        xmin_txt = doc.createTextNode(txt_xmin)   # 标注框的xmin
        xmin_node.appendChild(xmin_txt)
        # ymin
        ymin_node = doc.createElement('ymin')
        bndbox.appendChild(ymin_node)
        ymin_txt = doc.createTextNode(txt_ymin)  # 标注框的ymin
        ymin_node.appendChild(ymin_txt)
        # xmax
        xmax_node = doc.createElement('xmax')
        bndbox.appendChild(xmax_node)
        xmax_txt = doc.createTextNode(txt_xmax)   # 标注框的xmax
        xmax_node.appendChild(xmax_txt)
        # ymax
        ymax_node = doc.createElement('ymax')
        bndbox.appendChild(ymax_node)
        ymax_txt = doc.createTextNode(txt_ymax)  # 标注框的ymax
        ymax_node.appendChild(ymax_txt)

    ################ 文件保存
    #定义输出的xml文件路径和名称
    temp_xmlfile = xml_path + txt_file_name[:-4] + '.xml'
    try:
        with open(temp_xmlfile,'w',encoding='UTF-8') as fh:
     # 4.writexml()第一个参数是目标文件对象，第二个参数是根节点的缩进格式，第三个参数是其他子节点的缩进格式，
     # 第四个参数制定了换行格式，第五个参数制定了xml内容的编码。
            doc.writexml(fh, indent='', addindent='\t', newl='\n', encoding='UTF-8')
            print('写入xml OK!')
    except Exception as err:
        print('错误信息：{0}'.format(err))

In [7]:
image_path_abs = image_path + images_folder + '/' + temp_text_total[0][0]   # # 原始图片绝对路径——修改
xml_create_func(images_folder, image_name, image_path_abs, image_size_ys, temp_text_total)

写入xml OK!
